In [1]:
from src.dataset import Mimic3Dataset, padded_collate
from src.model import DST
import pdb
import numpy as np
import torch
import pytorch_lightning as pl
import pandas as pd
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split

#### True ATE

0.767 hours

### Outcome Regression

In [3]:
dataset_treated = Mimic3Dataset(".", intervention=True)
dataset_control = Mimic3Dataset(".", intervention=False)

In [4]:
path = "multirun/2022-09-08/22-00-41/7/lightning_logs/version_0/checkpoints/epoch=0-step=1517.ckpt"

In [5]:
model = DST.load_from_checkpoint(path)

/home/pchatha/.cache/pypoetry/virtualenvs/mimic-sc_0JIDI-py3.8/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (ConcordanceIndex). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_no_full_state`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)


In [6]:
collate_fn = lambda x: padded_collate(x, pad_index=-100)
def get_predictions(dataset):
    dl = torch.utils.data.DataLoader(dataset, collate_fn=collate_fn, batch_size=128)
    predictor = pl.Trainer(gpus=[5])
    predictions = predictor.predict(model, dataloaders=dl)
    return torch.cat(predictions)

In [7]:
predict_treated = get_predictions(dataset_treated)

/home/pchatha/.cache/pypoetry/virtualenvs/mimic-sc_0JIDI-py3.8/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:92: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/home/pchatha/.cache/pypoetry/virtualenvs/mimic-sc_0JIDI-py3.8/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

/home/pchatha/mimic/src/dataset.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["codes"] = torch.tensor(


In [8]:
predict_control = get_predictions(dataset_control)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Predicting: 0it [00:00, ?it/s]

#### Outcome Regression Estimate

In [9]:
tau = torch.full(predict_treated.shape, 16)
ey_x1 = torch.minimum(tau, predict_treated)
ey_x0 = torch.minimum(tau, predict_control)
(ey_x1 - ey_x0).mean()

tensor(0.3809)

### Propensity Score Weighing

In [11]:
df = pd.read_csv("data/mimic3_df_28.csv", index_col=[0,1])

df.head()

gender  stay_length  vent  hypertension  coronary_ath  \
subject_id hours_in                                                          
4          0              0    -0.360949     0             0             0   
           1              0    -0.360949     0             0             0   
           2              0    -0.360949     0             0             0   
           3              0    -0.360949     0             0             0   
           4              0    -0.360949     0             0             0   

                     atrial_fib  hematocrit  hemoglobin  platelets  \
subject_id hours_in                                                  
4          0                  0    0.076528   -0.111641  -0.187116   
           1                  0    0.076528   -0.111641  -0.187116   
           2                  0    0.076528   -0.111641  -0.187116   
           3                  0    0.076528   -0.111641  -0.187116   
           4                  0    0.076528   -0.111641  -0.187116   

                     mean blood pressure  treated  control  baseline_hazard  \
subject_id hours_in                                                           
4          0                    2.523323        1        0         0.001000   
           1                    0.082396        1        0         0.000951   
           2                    0.082396        1        0         0.000905   
           3                   -0.490692        1        0         0.000861   
           4                   -0.320892        1        0         0.000819   

                       hazard         q  survival_prob  survives  \
subject_id hours_in                                                
4          0         0.000596  0.999404       0.999404         1   
           1         0.000739  0.999261       0.998666         1   
           2         0.000703  0.999297       0.997964         1   
           3         0.000712  0.999288       0.997253         1   
           4         0.000665  0.999335       0.996590         1   

                    first_failure  censored  corrected_survival  
subject_id hours_in                                              
4          0                  NaN     False                   1  
           1                  NaN     False                   1  
           2                  NaN     False                   1  
           3                  NaN     False                   1  
           4                  NaN     False                   1

In [12]:
cols = ["gender", "vent", "hypertension", "coronary_ath", "atrial_fib", "hematocrit", "hemoglobin", "platelets", "mean blood pressure"]
df_flat = df[cols].groupby(level=0).mean() 

In [13]:
lr = LogisticRegressionCV(random_state=20, max_iter=100)
lr.fit(df_flat.drop(columns="vent"), df_flat["vent"])
lr.score(df_flat.drop(columns="vent"), df_flat["vent"])

0.7070870296474623

In [14]:
pi_x = lr.predict_proba(df_flat.drop(columns="vent"))[:, 1]

In [15]:
_tau = 16
surv_restr = df.groupby(level=0)["corrected_survival"].head(_tau)
y = surv_restr.groupby(level=0).sum()
t = df.groupby(level=0)["vent"].any().astype(int)

In [16]:
w1 = (t / pi_x).sum()
w2 = ((1-t) / (1 - pi_x)).sum()

In [17]:
p1 = (y * t) / pi_x

In [18]:
p2 = ((1 - t) * y) / (1 - pi_x)

(p1 - p2).mean()

(p1.sum() / w1 ) - (p2.sum() / w2)

0.28514830316372475

In [19]:
(p1 - p2).mean()

0.8174239532317583

### AIPW

In [21]:
b1 = ((t * y) / pi_x) - (((1 - t) * y) / (1 - pi_x))
b2 = (t - pi_x) / (pi_x * (1 - pi_x)) * ((1 - pi_x) * ey_x1.numpy() + pi_x * ey_x0.numpy())

In [22]:
b1.mean() - b2.mean()

0.48162234052044967